# 4장 : Database

## Databases in Flask

- Flask는 db를 지원하지 않음. 그래서 앱에 가장 잘 맞는 db를 선택할 수 있음.
relational model, NoSQL
- Flask extension (pip install 함)
1) Flask-SQLAlchemy : SQLAlchemy 패키지를 Flask 친화적으로 래핑한 것
(Object Relational Mapper = 테이블과 SQL 대신 클래스나 object, method 등으로 db를 다룰 수 있도록 해줌. 이러한 high-level 오퍼레이션들을 db 명령어로 바꿔주는 역할을 함)  
특히 SQLAlchemy는 많은 관계형 db를 지원함. 개발 시에는 서버가 필요하지 않은 SQLite를 이용하고, 운영 서버에 디플로이 할 때는 앱의 변경 없이 postgreSQL 서버를 선택할 수 있음


## Database Migrations

2. Flask-Migrate : SQLAlchemy의 db 마이그레이션 프레임워크인 'Alembic'의 Flask wapper임. 

## Flask-SQLAlchemy Configuration

- config.py 에 SQLALCHEMY_DATABASE_URI, SQLALCHEMY_TRACK_MODIFICATIONS(db변경이 일어날 때마다 app에 시그널을 보낼 것인지) 추가
- __init__.py에 db 인스턴스, db 마이그레이션 엔진 인스턴스를 추가함. 그리고 db구조를 정의하는 models 모듈을 import함.

## Database Models

- models.py 파일에 User 모델을 추가함
- database models : DB에 저장될 데이터는 클래스의 집합으로 표현됨. SQLAlchemy 내부의 ORM 레이어는 클래스들을 db table의 행들로 변환함.
- db에 비밀번호를 직접 보관하지 않고, 비밀번호 hash값을 보관함.
- \_\_repr\_\_ : python에게 이 클래스의 object를 어떻게 프린트할 지 알려줌. 디버깅할 때 유용함.

## Migration Repository 만들기

- Alembic : 스키마 변경 시에 마이그레이션을 가능하게 해줌
- migration repository : 마이그레이션 스크립트를 저장하고 있는 디렉토리. db 스키마 변경이 발생할 때마다 마이그레이션 스크립트가 레파지토리에 추가됨. 마이그레이션을 db에 적용할 때, 이 스크립트들을 실행함.
- flask db init 명령으로 레파지토리를 만듦 > 프로젝트 루트에 migrations 폴더가 생성됨. 이 파일들은 앱 소스코드 관리 대상에 포함되어야 함.

## 첫번째 Database Migration

- DB migration 방법 2가지(자동/수동)
- 자동 : Alembic이 db model과 현재 db 스키마를 비교하여 마이그레이션 스크립트를 만듦. flask db migrate 명령어가 이 역할을 함. 수행 후에는 migrations/versions 밑에 파일이 생기고, 파일 첫 줄에 메시지가 들어감.  이 파일도 소스 관리 대상에 포함됨. db 변경은 아직 일어나지 않고 스크립트만 만들어짐.

```
> flask db migrate -m "users table"
```
upgrade() : 마이그레이션 수행 <-> downgrade() : 삭제(Alembic이 older version으로 마이그레이션도 가능하게 함)  
  
```
> flask db upgrade
```
위 명령어를 수행해야 db에 스크립트가 반영됨. (여기서는 SQLite를 사용하기 때문에 upgrade 명령어가 db가 존재하지 않는다는 것을 알고 생성함 > 프로젝트 루트 경로에 app.db 파일이 생성되고, 이게 SQLite db임. 만약 SQLite가 아니라면, upgrade 실행 전에 db 서버 안에 db를 만들어야 함)  
  
Flask-SQLAlchemy에서 클래스명을 AddressAndPhone 으로 쓰면, 테이블명은 address_and_phone이 됨. 만약 테이블 이름을 설정하고 싶다면, 모델 클래스에 \_\_tablename\_\_ 속성을 넣으면 됨.

## Database Upgrade and Downgrade Workflow

- 개발에서 flask db migrate 로 새로운 스크립트를 생성하고 > 스크립트 리뷰하면서 자동 생성이 잘 되었는지 확인하고 > flask db upgrade로 실제 db에 반영하고 > 이 마이그레이션 스크립트를 소스 관리 대상에 포함시켜서 커밋함
- 이후에 운영 서버에 반영할 때는, 새로운 마이그레이션 스크립트가 포함된 소스코드를 가져와서 flask db upgrade 를 실행함. > Alembic이 운영db가 최근 버전의 스키마가 아닌 걸 발견하고, 이전 릴리스 이후에 생성된 모든  마이그레이션 스크립트를 실행함.
- flask db downgrade : 최신 마이그레이션을 undo함. 개발 중에 스크립트 생성하고, undo하고, 스크립트 지우는 용도로 주로 사용함. 

## Database Relationships

- models.py에 user 테이블과 post 테이블 간 relationship을 추가함.
- datetime.utcnow : UTC 시간을 사용하면, display될 때 사용자의 로컬 시간으로 변환될 것임
- db.ForeignKey('테이블명.필드명')  
Post 클래스의 user_id = db.Column(db.Integer, db.ForeignKey('user.id'))  
user 테이블의 id값을 참조함. 실제 db 컬럼이 생성됨

- db.relationship(''다'쪽의 모델 클래스명', backref=''다'쪽에서 '1'쪽을 참조할 때 사용할 이름')  
User 클래스의 posts = db.relationship('Post', backref='author', lazy='dynamic')  
1대 다 관계의 경우, db.relationship 필드는 '1'쪽에 정의되어, '다' 쪽에 접근하기 편한 길로 이용됨. db에 컬럼이 생성되지는 않지만, user 클래스에 posts라는 속성이 생기고, post클래스에 author라는 속성이 생김(가상 필드)
ex) u.posts 로 써서, 그 사용자가 작성한 모든 포스트를 가져옴.  
ex) post.author 로 그 포스트의 작성자(User)를 가져옴. 

- 특이사항 : ForeignKey 에서는 테이블명, relationship에서는 클래스명을 사용하기 때문에, 대소문자에 유의해야 함


## Playing with the Database

- db.session.add() : 세션에서 추가. 변경사항을 쌓아둠.
- db.session.delete() : 세션에서 삭제. 변경사항을 쌓아둠.
- db.session.commit() : 모든 변경사항들이 한 번에 등록됨.
- db.session.rollback() : 세션에서 작업 중에 에러가 발생하면, 이 함수가 불리면서 session을 abort하고, 모든 변경사항을 지움.


- <Class name>.query : 모든 모델은 query라는 속성을 가지고 있고, db 쿼리를 날릴 수 있는 진입점이 됨.  
    이하 함수들은 query. 뒤에 붙는 함수
- .all() : 클래스의 모든 엘리먼트를 반환
- .get(id) : 해당 id의 행을 반환
- .order_by(User.username.desc()).all() : username에 내림차순으로 정렬

## Shell Context

- flask shell : 이 명령어는 app 인스턴스를 미리 import하여 쉘 환경을 만듦.
- microblog.py에 @app.shell_context_processor 데코레이터를 등록함. 쉘 세션에 db 인스턴스와 모델들을 추가하여 shell context를 만드는 함수임. flask shell 명령어가 들어오면 이 함수가 invoke됨.
- 따라서 flask shell 쓰고, db 하면 import 없이 바로 db 인스턴스를 얻을 수 있음.  
※ 1장에서 프로젝트 탑레벨 디렉토리에 .flaskenv 파일을 만들어서  FLASK_APP=microblog.py 라는 환경변수를 지정해줌 > 그래서 flask 명령어를 칠 때마다 이 환경변수가 자동으로 등록되도록 함  
※ FLASK_APP : FLASK 애플리케이션을 구현한 파일명을 나타내는 환경변수.
- flask run > .flaskenv 파일 읽어서 microblog.py 파일을 실행시킴 > 첫 줄의 from app import app 하면서 app이 import되고, app/\_\_init\_\_.py 파일이 실행됨. > app = Flask(\_\_name\_\_) 으로 플라스크 앱 만듦